In [12]:
import numpy as np
from matplotlib import pyplot as plt
from pulp import *
from functools import partial
from collections import namedtuple

'选择上传ipynb是因为他可以保存上一次运行结果,老师可以直接查看我的结果.\n    因为pulp在windows平台上可能缺少一个支持库文件所以不一定能运行.\n    推荐将代码粘出在别处运行'

In [13]:
prob = LpProblem("Water_Transmission_Problem", LpMaximize)
#A
A = LpVariable("A_var", 0, 50, LpContinuous)
A1 = LpVariable("AJia", 0, None, LpContinuous)
A2 = LpVariable("AYi", 0, None, LpContinuous)
A3 = LpVariable("ABing", 0, None, LpContinuous)
A4 = LpVariable("ADing", 0, None, LpContinuous)
#B
B = LpVariable("B_var", 0, 60, LpContinuous)
B1 = LpVariable("BJia", 0, None, LpContinuous)
B2 = LpVariable("BYi", 0, None, LpContinuous)
B3 = LpVariable("BBing", 0, None, LpContinuous)
B4 = LpVariable("BDing", 0, None, LpContinuous)
#C
C = LpVariable("C_var", 0, None, LpContinuous)
C1 = LpVariable("CJia", 0, None, LpContinuous)
C2 = LpVariable("CYi", 0, None, LpContinuous)
C3 = LpVariable("CBing", 0, None, LpContinuous)
# 3. 设置目标函数
prob += 900*(A+B+C)-160*A1-130*A2-220*A3-170*A4-140*B1-130*B2-190*B3-150*B4-190*C1-200*C2-230*C3, "Total Cost of Ingredients per can"

# 4. 施加约束
prob += A - A1 - A2 - A3 - A4 == 0, "ABalance"
prob += B - B1 - B2 - B3 - B4 == 0, "BBalance"
prob += C - C1 - C2 - C3 == 0, "CBalance"
prob += A1+B1+C1 >= 30.0, "JiaSat"
prob += A2+B2+C2 >= 70.0, "YiSat"
prob += A3+B3+C3 >= 10.0, "BingSat"
prob += A4+B4 >= 10.0, "DingSat"
prob += A1+B1+C1 <= 80.0, "JiaLim"
prob += A2+B2+C2 <= 140.0, "YiLim"
prob += A3+B3+C3 <= 30.0, "BingLim"
prob += A4+B4 <= 50.0, "DingLim"
prob.solve()
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost of Ingredients per can = ", value(prob.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/summage/.local/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/6dafdb31314d47e0ab47823a81c3073e-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/6dafdb31314d47e0ab47823a81c3073e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 67 RHS
At line 79 BOUNDS
At line 82 ENDATA
Problem MODEL has 11 rows, 14 columns and 36 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (-4) rows, 13 (-1) columns and 23 (-13) elements
0  Obj 20929.998 Primal inf 230.1 (4) Dual inf 3880 (5)
10  Obj 216600
Optimal - objective value 216600
After Postsolve, objective 216600, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 216600 - 10 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.

In [14]:
prob = LpProblem("Oil_Production_Problem", LpMaximize)

Jia = LpVariable('Jia_var', 0., None, cat=LpContinuous)
AJia = LpVariable('AJia_var', 0., None, cat=LpContinuous)
BJia = LpVariable('BJia_var', 0., None, cat=LpContinuous)
Yi = LpVariable('Yi_var', 0., None, cat=LpContinuous)
BYi = LpVariable('BYi_var', 0., None, cat=LpContinuous)
AYi = LpVariable('AYi_var', 0., None, cat=LpContinuous)
A1 = LpVariable('A1_0-500', 0., 500., cat=LpContinuous)
w1 = LpVariable('A1State', 0, 1, cat=LpInteger)
A2 = LpVariable('A2_500-1000', 0., 500., cat=LpContinuous)
w2 = LpVariable('A2State', 0, 1, cat=LpInteger)
A3 = LpVariable('A3_1000-1500', 0., 500., cat=LpContinuous)
w3 = LpVariable('A3State', 0, 1, cat=LpInteger)
        
prob += 4800*Jia + 5600*Yi - 10000*A1 - 8000*A2 - 6000*A3, "Total Cost of Ingredients per can"

prob += 500 + A1 + A2 + A3 - AJia - AYi >= 0.0, "ALeft"
prob += 1000 - BJia  - BYi >= 0.0, "BLeft"
prob += Jia - AJia - BJia == 0.0, "JiaBalanced"
prob += AJia - 0.5*Jia >= 0.0, "JiaWellPortioned"
prob += Yi - AYi - BYi == 0.0, "YiBalanced"
prob += AYi - 0.6*Yi >= 0.0, "YiWellPortioned"
# prob += AJia + AYi- 500.0*w1 >= 0.0, "A1Needed"
prob += 500*w1 - A1 >= 0.0, "A1Correct"
prob += 500*w2 - A1 <= 0.0, "A2Needed"
prob += 500*w2 - A2 >= 0.0, "A2Correct"
prob += 500*w3 - A2 <= 0.0, "A3Needed"
prob += 500*w3 - A3 >= 0.0, "A3Correct"
prob.solve(use_mps=False)
# print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total Cost of Ingredients per can = ", value(prob.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/summage/.local/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/f9e49f9f84174c59b9a868e1b59f2341-pulp.lp timeMode elapsed branch printingOptions all solution /tmp/f9e49f9f84174c59b9a868e1b59f2341-pulp.sol (default strategy 1)
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Option for timeMode changed from cpu to elapsed
Continuous objective value is 6e+06 - 0.00 seconds
Cgl0004I processed model has 7 rows, 8 columns (2 integer (2 of which binary)) and 20 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.666667
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. -5e+06 iterations 2
Cbc0038I Solution found of -5e+06
Cbc0038I Relaxing continuous gives -5e+06
Cbc0038I Before mini branch and bound, 0 integers at bound fixed and 3 continuous
Cbc0038I Full problem 7 rows 8 colu

In [35]:
prob = LpProblem("Cut_Problem", LpMaximize)
mod1 = LpVariable("mod1_var", 0, None, LpInteger)
mod2 = LpVariable("mod2_var", 0, None, LpInteger)
mod3 = LpVariable("mod3_var", 0, None, LpInteger)
mod4 = LpVariable("mod4_var", 0, 20000, LpInteger)
can = LpVariable("can_var", 0, None, LpInteger)
waste = LpVariable("waste_var", 0, None, LpContinuous)
# 3. 设置目标函数
prob += 0.1*can-0.001*waste, "Net_income"

# 4. 施加约束
prob += 40*60*60 - 1.5*mod1 - 2*mod2 - 1*mod3-3*mod4 >= 0, "Time"
prob += can - mod1 - 2 * mod2 - 4*mod4 <= 0, "Wrap"
prob += 2*can - 10*mod1-4*mod2-16*mod3-5*mod4 <= 0, "Bottom"
prob += mod1+mod2+mod3 <= 50000, "Total"
prob += (mod1+mod2+mod3)*24*24 + 32*28*mod4 - can*(2*3.14*6.25+5*3.14*10) - waste == 0.0 , "Waste"
prob.solve()
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost of Ingredients per can = ", value(prob.objective))
#

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/summage/.local/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/0a2d5d6f414248559bd38b2190f61d70-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/0a2d5d6f414248559bd38b2190f61d70-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 45 RHS
At line 51 BOUNDS
At line 57 ENDATA
Problem MODEL has 5 rows, 6 columns and 22 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 4282.06 - 0.00 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 5 rows, 6 columns (5 integer (0 of which binary)) and 22 elements
Cbc0012I Integer solution of -4282.0625 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective -4282.062499999996, to